## Configs

In [1]:
offset = 0
limit = 3000
period = 'max' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Download all NASDAQ traded symbols

In [2]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 12071


## Download Historic data

In [4]:
import yfinance as yf
import os, contextlib

In [5]:
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/us_stocks'

In [7]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            try:
                s = symbols[i]
                data = yf.download(s, period=period)
                if len(data.index) == 0:
                    continue
            
                is_valid[i] = True
                data.to_csv('{}/{}.csv'.format(DATA_PATH, s))
            except:
                pass

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 2476
CPU times: user 2min 51s, sys: 7.72 s, total: 2min 58s
Wall time: 52min 49s


In [8]:
valid_data = data_clean[is_valid]
valid_data.to_csv(f'{DATA_PATH}/symbols_valid_meta.csv', index=False)

## Separating ETFs and Stocks

In [9]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [14]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join(DATA_PATH, filename), join(dest, filename))
        
# move_symbols(etfs, f"{DATA_PATH}/etfs")
move_symbols(stocks, f"{DATA_PATH}/stocks")